In [ ]:
# grab the functions for making the measurements with the group data
include("../src/obs_funcs.jl")
import Jackknife.get_sdss_subvols

In [ ]:
# build the data sample from the group finding catalog
# the function also finds the sample volumes from the redshift range
galdf, samplevols = load_groupdat()

In [ ]:
# compute the jackknife subvolumes that our selected sample belongs to
radecs = zeros(size(galdf)[1], 2)
radecs[:, 1] = rad2deg.(galdf[:ra])
radecs[:, 2] = rad2deg.(galdf[:dec])

galdf[:jackvol] = get_sdss_subvols(radecs)

In [ ]:
# define the galaxy mass samples with the following bin edges in log10(M*)
const nmbins = 4
const logMedges = [9.4, 9.8, 10.3, 10.6, 11.0]
const mlabels = [9.7, 10.1, 10.4, 10.7]  # these values are closest to the bin means/medians
const meanMs = Array{Float64}(nmbins)

for i in 1:nmbins
    meanM = mean(galdf[:log10M][logMedges[i] .<= galdf[:log10M] .< logMedges[i + 1]])
    meanMs[i] = meanM
end

In [ ]:
# similarly, define the bin edges in log10(δ + 1)
const ndbins = 8
const dbinedges = Array(linspace(-0.8, 1.2, 9))
const dmeans = Array{Float64}(4, 8)

for i in 1:nmbins
    massdf = galdf[logMedges[i] .<= galdf[:log10M] .< logMedges[i + 1], :]
    for j in 1:ndbins
        meand = mean(massdf[:logρ][dbinedges[j] .<= massdf[:logρ] .< dbinedges[j + 1]])
        dmeans[i, j] =  meand
    end
end

In [ ]:
dbinedges

In [ ]:
# now to plot some variations in galaxy properties
# start with the scale radius
galdf[:Rₑ] = galdf[:R_e]
yticks = [Array(-0.25:0.05:0.15), Array(-0.1:0.05:0.2), Array(-0.1:0.05:0.2), Array(-0.1:0.05:0.2)]
annotatevals = [[-1, 3], [0, 6], [0, 6], [0, 6]]
replot = massenv_plot(galdf, "Rₑ",
                      yticks, annotatevals,
                      dbinedges, dmeans,
                      samplevols[1],
                      ptcolor = "orange")

In [ ]:
subdf = galdf[logMedges[1] .<= galdf[:log10M] .< logMedges[1 + 1], :]
col = "Rₑ"
cencol = Symbol(string("cen", col))
subdf[cencol] = rm_mean_logmass(subdf[Symbol(col)], subdf[:log10M])
subdf[cencol] = subdf[cencol] / mean(subdf[Symbol(col)])

In [ ]:
subarr = subdf[find(Array(subdf[:jackvol]) .== 1), :]

In [ ]:
envmeans(subarr, (cencol, dbinedges,)...)

In [ ]:
sum(ans)